# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 592


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [3]:
# Set the API base URL
url = f'https://api.openweathermap.org/data/2.5/weather?'
units = "imperial"
city_name = "city"
city_url = url + "&appid=" + weather_api_key + "&q=" + city_name + "&units=" + units


In [4]:
requests.get(city_url).json()
# response =requests.get(city_url).json()
# response

{'coord': {'lon': 149.1292, 'lat': -35.2813},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 46.15,
  'feels_like': 46.15,
  'temp_min': 44.58,
  'temp_max': 48.9,
  'pressure': 1016,
  'humidity': 95,
  'sea_level': 1016,
  'grnd_level': 939},
 'visibility': 10000,
 'wind': {'speed': 1.14, 'deg': 10},
 'clouds': {'all': 75},
 'dt': 1723935038,
 'sys': {'type': 1,
  'id': 9588,
  'country': 'AU',
  'sunrise': 1723927270,
  'sunset': 1723966432},
 'timezone': 36000,
 'id': 2172517,
 'name': 'City',
 'cod': 200}

In [ ]:
# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = url + "&appid=" + weather_api_key + "&q=" + city_name + "&units=" + units

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

    # pause to avoid rate limiting
    # time.sleep(1)

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | taltal
Processing Record 2 of Set 1 | waitangi
Processing Record 3 of Set 1 | breckenridge
Processing Record 4 of Set 1 | ribeira grande
Processing Record 5 of Set 1 | narsingi
Processing Record 6 of Set 1 | rio grande
Processing Record 7 of Set 1 | anton lizardo
Processing Record 8 of Set 1 | uzgen
Processing Record 9 of Set 1 | nizhneangarsk
Processing Record 10 of Set 1 | cuito
Processing Record 11 of Set 1 | nakamura
Processing Record 12 of Set 1 | sitka
Processing Record 13 of Set 1 | vila franca do campo
Processing Record 14 of Set 1 | stanley
Processing Record 15 of Set 1 | gordonvale
Processing Record 16 of Set 1 | vilyuchinsk
Processing Record 17 of Set 1 | meadow lakes
Processing Record 18 of Set 1 | port-aux-francais
Processing Record 19 of Set 1 | longyearbyen
Processing Record 20 of Set 1 | tiksi
Processing Record 21 of Set 1 | manismata
Processing Record 22 of Set 1 | adamstown
Proc

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
latitude = city_data_df["Lat"]
temperature = city_data_df["Max Temp"]
x_axis = latitude
y_axis = temperature
plt.scatter(x_axis, y_axis, facecolors="lightblue", edgecolors="black")

# Incorporate the other graph properties
plt.title("City Max Latitude vs. Temperature(2022-10-18)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (C)")
plt.grid()

# Save the figure
plt.savefig("output_data/Lat_vs_Temperature.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
latitude = city_data_df["Lat"]
humidity = city_data_df["Humidity"]
x_axis = latitude
y_axis = humidity
plt.scatter(x_axis, y_axis, facecolors="lightblue", edgecolors="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity (2022-10-18)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid()

# Save the figure
plt.savefig("output_data/Lat_vs_Humidity.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
latitude = city_data_df["Lat"]
cloudiness = city_data_df["Cloudiness"]
x_axis = latitude
y_axis = cloudiness
plt.scatter(x_axis, y_axis, facecolors="lightblue", edgecolors="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Cloudiness (2022-10-18)")
plt.xlabel("Latitude")
plt.ylabel("cloudiness(%)")
plt.grid()

# Save the figure
plt.savefig("output_data/Lat_vs_Cloudiness.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
latitude = city_data_df["Lat"]
wind_speed = city_data_df["Wind Speed"]
x_axis = latitude
y_axis = wind_speed
plt.scatter(x_axis, y_axis, facecolors="lightblue", edgecolors="black")

# Incorporate the other graph properties
plt.title("City Latitude vs. Wind Speed (2022-10-18)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (m/s)")
plt.grid()

# Save the figure
plt.savefig("output_data/Lat_vs_Windspeed.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
def linear_regression_plot(df, x_col, y_col, title, color="lightblue"):
    x_data = df[x_col]
    y_data = df[y_col]

    linear_regression = linregress(x_data, y_data)
    # print(linear_regression)
    y_regression = x_data * linear_regression.slope + linear_regression.intercept  #mx +b =y (slop=m, b=intercept, x=value)
    # print(y_regression)
    
    plt.scatter(x_data, y_data, facecolors=color, edgecolors="black")
    
    linear_equation = "y = " + str(round(linear_regression.slope,2)) + "x + " + str(round(linear_regression.intercept,2))
    annotation_x = x_data.min()
    annotation_y = y_data.max()
    
    plt.plot(x_data, y_regression, "-", color="red")
    plt.annotate(linear_equation, xy=(annotation_x, annotation_y), color="red")
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title(title)
    print(f"The r-value is: {linear_regression.rvalue}")
    plt.show()

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df.loc[city_data_df["Lat"] >= 0]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df.loc[city_data_df["Lat"] < 0]

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
linear_regression_plot(northern_hemi_df, "Lat", "Max Temp", "Northern Hemisphere: Temperature vs. Latitude", color="lightblue")

In [ ]:
# Linear regression on Southern Hemisphere
linear_regression_plot(southern_hemi_df, "Lat", "Max Temp", "Southern Hemisphere: Temperature vs. Latitude", color="lightblue")

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
linear_regression_plot(northern_hemi_df, "Lat", "Humidity", "Northern Hemisphere: Humidity vs. Latitude", color="lightblue")

In [ ]:
# Southern Hemisphere
linear_regression_plot(southern_hemi_df, "Lat", "Humidity", "Southern Hemisphere: Humidity vs. Latitude", color="lightblue")

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
linear_regression_plot(northern_hemi_df, "Lat", "Cloudiness", "Northern Hemisphere: Cloudiness vs. Latitudes", color="lightblue")

In [ ]:
# Southern Hemisphere
linear_regression_plot(southern_hemi_df, "Lat", "Cloudiness", "Southern Hemisphere: Cloudiness vs. Latitude", color="lightblue")

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
linear_regression_plot(northern_hemi_df, "Lat", "Wind Speed", "Northern Hemisphere: Wind Speed vs. Latitude", color="lightblue")

In [ ]:
# Southern Hemisphere
linear_regression_plot(southern_hemi_df, "Lat", "Wind Speed", "Southern Hemisphere: Wind Speed vs. Latitude", color="lightblue")

**Discussion about the linear relationship:** YOUR RESPONSE HERE